In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir


In [ ]:

# dir = 'datasets/CASIA_Iris_Image_Database_(version1.0)/001/1'
        
# test = cv.imread(os.path.join(dir, '001_1_2.bmp'))

test = cv.imread('001_1_1.bmp')
plt.imshow(test)

test = cv.cvtColor(test, cv.COLOR_BGR2GRAY)
test = cv.equalizeHist(test)
plt.imshow(test, cmap="gray")


# LOCALIZATION
test = cv.medianBlur(test,5)
edges = cv.Canny(test,50,200)

plt.imshow(edges, cmap = "gray")

circles = cv.HoughCircles(edges,cv.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=10,maxRadius=70)

print(circles)

# saving iris and pupil values 
# circles = np.uint16(np.around(circles))
# circles
# for i in circles[0,:]:
#     centerX = i[0]
#     centerY = i[1]
#     radius = i[2]
#     iris = (centerX, centerY, (radius * 2))
#     pupil = (centerX, centerY, radius)
#     # draw the outer circle
#     # cv.circle(test,(iris[0], iris[1]), iris[2],(0,255,0),2)
#     cv.circle(test,(pupil[0], pupil[1]), pupil[2],(0,255,0),2)
#     # print(pupil)
    


# circles2 = cv.HoughCircles(edges,cv.HOUGH_GRADIENT, 1, pupil[2] * 2,
#                             param1=50,param2=30,minRadius=0,maxRadius=0)
#                            # param2=150)

# circles2 = np.uint16(circles2)
# circles2
# for i in circles2[0,:]:
#     # draw the outer circle
#     cv.circle(test,(i[0],i[1]),i[2],(0,255,0),2)
#     # center = (pupil[0], pupil[1])
#     # radius = i[2]
#     # mask = np.zeros((test.shape[0], test.shape[1], 1), np.uint8)
#     # cv.circle(mask, center, radius, (255, 255, 255), thickness = -1)
#     # # Mask the iris and crop everything outside of its radius.
#     # test = cv.bitwise_and(test, mask)

    
cv.imshow('detected circles',test)
cv.waitKey(0)
cv.destroyAllWindows()